In [1]:
from langchain_core.documents import Document

document = [
    Document(
        page_content="Dogs are great companions, Known for their loyalty and friendliness.",
        metadata= {"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata= {"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata= {"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Parrot are intelligent birds capable of mimicking human speech.",
        metadata= {"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata= {"source": "mammal-pets-doc"},
    ),
]

In [2]:
document

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, Known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Parrot are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()


groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017CF3ACD7D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017CF3C1D290>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
# VectorStors
from langchain_chroma import Chroma

vectorstor = Chroma.from_documents(documents=document, embedding=embeddings)
vectorstor

In [6]:
vectorstor.similarity_search("cat")

[Document(id='0375140c-b8b5-403b-83ad-ae1a35364ce8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='4cad1bf1-ab7a-4512-b72f-c5f9ae77944c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, Known for their loyalty and friendliness.'),
 Document(id='23ce12ba-defd-40e6-a861-90bc5227b8d5', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='f924e4c6-a2cb-47d5-94a2-0fb2dbcf5c49', metadata={'source': 'mammal-pets-doc'}, page_content='Parrot are intelligent birds capable of mimicking human speech.')]

In [7]:
### Async Query
await vectorstor.asimilarity_search("cat")

[Document(id='0375140c-b8b5-403b-83ad-ae1a35364ce8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='4cad1bf1-ab7a-4512-b72f-c5f9ae77944c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, Known for their loyalty and friendliness.'),
 Document(id='23ce12ba-defd-40e6-a861-90bc5227b8d5', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='f924e4c6-a2cb-47d5-94a2-0fb2dbcf5c49', metadata={'source': 'mammal-pets-doc'}, page_content='Parrot are intelligent birds capable of mimicking human speech.')]

In [8]:
vectorstor.similarity_search_with_score("cat")


[(Document(id='0375140c-b8b5-403b-83ad-ae1a35364ce8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.935105562210083),
 (Document(id='4cad1bf1-ab7a-4512-b72f-c5f9ae77944c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, Known for their loyalty and friendliness.'),
  1.5740901231765747),
 (Document(id='23ce12ba-defd-40e6-a861-90bc5227b8d5', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956904888153076),
 (Document(id='f924e4c6-a2cb-47d5-94a2-0fb2dbcf5c49', metadata={'source': 'mammal-pets-doc'}, page_content='Parrot are intelligent birds capable of mimicking human speech.'),
  1.6487725973129272)]

In [10]:
### Retrivers
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriver = RunnableLambda(vectorstor.similarity_search).bind(k=1)
retriver.batch(["cat","dog"])

[[Document(id='0375140c-b8b5-403b-83ad-ae1a35364ce8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='4cad1bf1-ab7a-4512-b72f-c5f9ae77944c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, Known for their loyalty and friendliness.')]]

In [12]:
retrivers =vectorstor.as_retriever(
    search_type = "similarity",
    search_kwargs= {"k": 1}
)
retriver.batch(["cat", "Dog"])

[[Document(id='0375140c-b8b5-403b-83ad-ae1a35364ce8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='4cad1bf1-ab7a-4512-b72f-c5f9ae77944c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, Known for their loyalty and friendliness.')]]

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

messages = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages(["human", messages])


rag_chain = {"context": retriver, "question": RunnablePassthrough()}|prompt|model

response = rag_chain.invoke("tell me about dogs")

response.content

'Dogs are great companions, known for their loyalty and friendliness.'